In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import dtale

In [2]:
pd.set_option('display.max_columns', None)
%matplotlib inline

In [3]:
def convert_dtype(x):
    if not x:
        return ''
    try:
        return str(x)
    except:
        return ''
def convert_dtypea(x):
    if not x:
        return ''
    try:
        return int(x)
    except:
        return ''
    
smile = pd.read_csv('../data/Smileon.2021.10.17.csv', skiprows=2, header=0, encoding='latin-1', converters={
     'LinkedToCallReportNum':convert_dtype,'ThirdPartyStateProvince':convert_dtype, 'FeedbackFromPhoneWorkerName':convert_dtype,
'SMILE ON 60+ Demographic Information - # of People in Household':convert_dtypea, 'Birth Year':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 2393 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - 2950 - Quantity':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 4342 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - 7140 - Quantity':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 7320 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - Lab Assist narrative':convert_dtype, 'SMILE ON 60+ Case Management Information - What is the outcome for the client?':convert_dtype,
'SMILE ON 60+ Case Management Information - Dental Clinic':convert_dtype})

In [4]:
df = smile

In [5]:
df = df.drop(columns = 'SMILE ON 60+ Base-line Oral and Conditions Questions - Dental Clinic')
df = df.drop(columns = 'SMILE ON 60+ Case Management Information - Dental Clinic')
df = df.drop(columns = 'SMILE ON 60+ Presumptive Eligibility - What is the next step for clients encounter?')
df = df.drop(columns = 'SMILE ON 60+ Oral Care Encounter - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)')

In [6]:
##used this for general data cleaning
df.columns = [x.replace('SMILE ON 60+ Demographic Information -','') for x in df.columns]
df.columns = [x.replace(' - Quantity','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Screening -','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Base-line Oral and Conditions Questions - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Case Management Information - ','') for x in df.columns]
df.columns = [x.replace('(Select all that apply)','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Case Management Information - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Demographic Information - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Screening -','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Clinic Information - ','') for x in df.columns]

In [16]:
df_clean = df.drop(columns = ['ReportVersion','LinkedToCallReportNum','CallDateAndTimeEnd','CallLength',
            'PhoneWorkerNum','WasRealCall','WasHangup','WasSexCall','WasWrongNumber',
            'WasPrankCall','WasSilentCall','GeoCode','GeoAssignment','CensusDivision','CensusTrack',
            'CensusReportingArea','211Region','PhoneType','ThirdPartyName','ThirdPartyOrganization',
            'ThirdPartyPhoneNumber','ThirdPartyAddress','ThirdPartyCity','ThirdPartyCounty',
            'ThirdPartyStateProvince','ThirdPartyPostalCode','Narrative','VolunteerComments',
            'Feedback','CallersFeedback','TextField2','TextField3','TextField4','TextField5',
            'TextField6','TextField7','TextField8','TextField9','TextField10','EnteredByWorkerNum',
            'EnteredOn','Supervisor','Reviewed','FeedbackStatus','FeedbackFromPhoneWorkerNum',
             'FeedbackFromPhoneWorkerName','ReferralsMade','IPAddress','OrgNum','VolIdentifier',
            'SMILE ON 60+ Verification of Eligibility - Verification Date',
            'Did the client commit to referral?','Did you help in these other areas?',
             'Transportation Barriers','Action: Upload Before Picture on Finish Tab','Was initial appointment made?',
            'Was initial transportation assistance or referral provided?',
            'SMILE ON 60+ Oral Care Encounter - Action: Upload After Picture on Finish Tab',
            'SMILE ON 60+ Presumptive Eligibility - Dental Insurance?','SMILE ON 60+ Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?',
            'SMILE ON 60+ Site Information - Enroller ID',
            'SMILE ON 60+ Site Information - Enroller Location',
            'SMILE ON 60+ Verification of Eligibility - Transportation Arranged','SMILE ON 60+ Verification of Eligibility - Verification of Eligibility',
            'SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab',' Root Fragments',' Need for Periodontal Care',' Suspicious Soft Tissue Lesions',
            ' Untreated Decay',' What other assistance programs are you on?',' Preferred Language/Language Needed',
            ' How did you first hear about the SMILE ON 60+ program?','SMILE ON 60+ Presumptive Eligibility - What is the next step for clients encounter? ',
            'What is the next step for clients encounter?'])

In [17]:
df_clean.columns = [x.replace('SMILE ON 60+ Oral Care Encounter -','') for x in df_clean.columns]
df_clean.columns = [x.replace('Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)','social_6_restored') for x in df_clean.columns]
df_clean.columns = [x.replace('SMILE ON 60+ Presumptive Eligibility -','') for x in df_clean.columns]
df_clean.columns = [x.replace('What barriers have kept you from finding a consistent dental home?','barriers') for x in df_clean.columns]
df_clean.columns = [x.replace('How would you describe the condition of your mouth and teeth?','mouth_condition') for x in df_clean.columns]
df_clean.columns = [x.replace('Have you ever felt that the appearance of your mouth and teeth affected your quality of life?','teeth_affect_qualifty_of_life') for x in df_clean.columns]
df_clean.columns = [x.replace('No Code - ','') for x in df_clean.columns]
df_clean.columns = [x.replace('Experienced any problems related to mouth/teeth during past 12 months?','problems_past_year') for x in df_clean.columns]
df_clean.columns = [x.replace('How often do you brush your teeth?','brush_frequency') for x in df_clean.columns]


In [18]:
df_clean.head(1)

,CallReportNum,CallDateAndTimeStart,CallerNum,CityName,CountyName,StateProvince,CountryName,PostalCode,# of People in Household,Birth Year,Other Language,SMILE ON 60+ Notes (Optional) - Notes,1351,1354,2140,2150,2160,2161,220,230,2330,2331,2332,2335,2390,2391,2392,2393,2394,2915,2920,2933,2940,2950,2954,2955,2980,3220,4341,4342,6930,6980,7111,7140,7210,7220,7230,7240,7241,7250,7261,7285,7286,7310,7311,7320,7321,7410,7411,7412,7413,7414,7415,7440,7450,7451,7460,7461,7471,7510,7511,7970 - Mandibular Quantity,7970 - Maxillary Quantity,7971,7972,9110,9951,Lab Assist narrative,Denture Step Mandibular Quantity,Denture Step Maxillary Quantity,Last Screening Date,Are you limited in what you can eat?,Do you need help in these other areas?,teeth_affect_qualifty_of_life,mouth_condition,What is the next step for client encounter?,barriers,What is the outcome for the client?,Is patient of record date if yes last seen?,Gender,Household Income Level,Race/Ethnicity,When is the last time you visited your dentist?,Is the treatment plan completed?,Was a treatment plan developed?,What care was provided to the enrollee?,# of Lower Natural Teeth,# of Upper Natural Teeth,Do you have removable dentures?,Do you usually wear your denture(s) when you eat?,problems_past_year,brush_frequency,Treatment Urgency,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,SMILE ON 60+ Treatment Plan - social_6_restored
0,93115324,9/30/2021 16:56,3638169,Smyrna,Rutherford,TN,United States,37167.0,1,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,Yes,No,Yes,Poor,NaN,No insurance,,NaN,Female,$0 - $21 280,African American,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
d = dtale.show(df_clean)
d.open_browser()

2021-11-13 12:00:14,024 - INFO     - NumExpr defaulting to 4 threads.


In [88]:
urgent = df_clean.loc[df_clean[' Treatment Urgency'] == 'Urgent Care - within next week-pain or infection']
urgent = urgent.filter(['CallReportNum','CallerNum'])

In [89]:
urgent.head(1)

,CallReportNum,CallerNum
545,92735826,3518717


In [90]:
urgent_merged = df_clean.merge(urgent,how='inner',on='CallerNum')

In [98]:
urgent_merged.tail(3)

,CallReportNum_x,CallDateAndTimeStart,CallerNum,CityName,CountyName,StateProvince,CountryName,PostalCode,# of People in Household,Birth Year,Other Language,SMILE ON 60+ Notes (Optional) - Notes,1351,1354,2140,2150,2160,2161,220,230,2330,2331,2332,2335,2390,2391,2392,2393,2394,2915,2920,2933,2940,2950,2954,2955,2980,3220,4341,4342,6930,6980,7111,7140,7210,7220,7230,7240,7241,7250,7261,7285,7286,7310,7311,7320,7321,7410,7411,7412,7413,7414,7415,7440,7450,7451,7460,7461,7471,7510,7511,7970 - Mandibular Quantity,7970 - Maxillary Quantity,7971,7972,9110,9951,Lab Assist narrative,Denture Step Mandibular Quantity,Denture Step Maxillary Quantity,Last Screening Date,Are you limited in what you can eat?,Do you need help in these other areas?,teeth_affect_qualifty_of_life,mouth_condition,What is the next step for client encounter?,barriers,What is the outcome for the client?,Is patient of record date if yes last seen?,Gender,Household Income Level,Race/Ethnicity,When is the last time you visited your dentist?,Is the treatment plan completed?,Was a treatment plan developed?,What care was provided to the enrollee?,# of Lower Natural Teeth,# of Upper Natural Teeth,Do you have removable dentures?,Do you usually wear your denture(s) when you eat?,problems_past_year,brush_frequency,Treatment Urgency,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,SMILE ON 60+ Treatment Plan - social_6_restored,CallReportNum_y
4660,56184279,9/5/2018 13:06,2419440,Cleveland,Bradley,TN,United States,37311.0,1,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,9/4/2018,Yes,NaN,Yes,Poor,NaN,Dental cost; No insurance,,Less than 12 months,Female,$0 - $21 280,Caucasian,Less than 12 months,NaN,NaN,D 0191 - Assessment of a patient - 15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,55980337
4661,55980337,9/4/2018 8:13,2419440,Cleveland,Bradley,TN,United States,37311.0,1,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,9/4/2018,Yes,NaN,Yes,Poor,NaN,Dental cost; No insurance,,Less than 12 months,Female,$0 - $21 280,Caucasian,Less than 12 months,NaN,NaN,7140 - Extraction - erupted tooth or exposed r...,13.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,twice daily,Urgent Care - within next week-pain or infection,No,Yes,NaN,55980337
4662,55979799,9/4/2018 7:47,2419440,Cleveland,Bradley,TN,United States,37311.0,1,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,Yes,No,Yes,Poor,NaN,Dental cost; No insurance,,NaN,Female,$0 - $21 280,Caucasian,Less than 12 months,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55980337


In [99]:
urgent_merged_2419440 = urgent_merged.loc[urgent_merged['CallerNum']== 2419440]

In [100]:
urgent_merged_2419440

,CallReportNum_x,CallDateAndTimeStart,CallerNum,CityName,CountyName,StateProvince,CountryName,PostalCode,# of People in Household,Birth Year,Other Language,SMILE ON 60+ Notes (Optional) - Notes,1351,1354,2140,2150,2160,2161,220,230,2330,2331,2332,2335,2390,2391,2392,2393,2394,2915,2920,2933,2940,2950,2954,2955,2980,3220,4341,4342,6930,6980,7111,7140,7210,7220,7230,7240,7241,7250,7261,7285,7286,7310,7311,7320,7321,7410,7411,7412,7413,7414,7415,7440,7450,7451,7460,7461,7471,7510,7511,7970 - Mandibular Quantity,7970 - Maxillary Quantity,7971,7972,9110,9951,Lab Assist narrative,Denture Step Mandibular Quantity,Denture Step Maxillary Quantity,Last Screening Date,Are you limited in what you can eat?,Do you need help in these other areas?,teeth_affect_qualifty_of_life,mouth_condition,What is the next step for client encounter?,barriers,What is the outcome for the client?,Is patient of record date if yes last seen?,Gender,Household Income Level,Race/Ethnicity,When is the last time you visited your dentist?,Is the treatment plan completed?,Was a treatment plan developed?,What care was provided to the enrollee?,# of Lower Natural Teeth,# of Upper Natural Teeth,Do you have removable dentures?,Do you usually wear your denture(s) when you eat?,problems_past_year,brush_frequency,Treatment Urgency,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,SMILE ON 60+ Treatment Plan - social_6_restored,CallReportNum_y
4660,56184279,9/5/2018 13:06,2419440,Cleveland,Bradley,TN,United States,37311.0,1,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,9/4/2018,Yes,NaN,Yes,Poor,NaN,Dental cost; No insurance,,Less than 12 months,Female,$0 - $21 280,Caucasian,Less than 12 months,NaN,NaN,D 0191 - Assessment of a patient - 15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,55980337
4661,55980337,9/4/2018 8:13,2419440,Cleveland,Bradley,TN,United States,37311.0,1,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,9/4/2018,Yes,NaN,Yes,Poor,NaN,Dental cost; No insurance,,Less than 12 months,Female,$0 - $21 280,Caucasian,Less than 12 months,NaN,NaN,7140 - Extraction - erupted tooth or exposed r...,13.0,7.0,No,No,Difficulty when biting or chewing foods; Dry m...,twice daily,Urgent Care - within next week-pain or infection,No,Yes,NaN,55980337
4662,55979799,9/4/2018 7:47,2419440,Cleveland,Bradley,TN,United States,37311.0,1,1952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,Yes,No,Yes,Poor,NaN,Dental cost; No insurance,,NaN,Female,$0 - $21 280,Caucasian,Less than 12 months,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55980337


In [101]:
urgent_merged.value_counts('CallReportNum_x')

CallReportNum_x
53192452    3
67860555    3
67861142    3
67861069    3
67860975    3
           ..
65029871    1
65045002    1
65045264    1
65045724    1
93492749    1
Length: 4272, dtype: int64